<img src="logo_UNSAM.jpg" align="right" width="120">


# <center>Trabajo final: Análisis y procesamiento de señales: *Detección de crisis epilépticas mediante análisis de EEG*</center> 

### <center> - Alumna: Lola Pampin</center> 

### <center> - Profesor: Mariano Llamedo Soria</center> 

### <center>Febrero 2026</center> 

## <center> Universidad Nacional de San Martín, Escuela de Ciencia y Tecnología</center> 




Dadas dos series temporales $x_h(t)$ y $x_k(t)$ (por ejemplo, series relacionadas con dos señales de EEG) y una ventana de tiempo $\Delta t\$ que contiene *N* instantes de tiempo, PLI y WPLI pueden definirse de la siguiente manera:

$$
PLI_{h,k,\Delta t} = \left| \frac{1}{N} \sum_{p=1}^{N} \text{sign}(\phi_h(p) - \phi_k(p)) \right|
$$

$$
WPLI_{h,k,\Delta t} = \frac{\left| \frac{1}{N} \sum_{p=1}^{N} \sin(\phi_h(p) - \phi_k(p)) \right|}{\frac{1}{N} \sum_{p=1}^{N} \left| \sin(\phi_h(p) - \phi_k(p)) \right|}
$$

donde $\phi_h(p)$ y $\phi_k(p)$ son las fases en el instante de tiempo 𝑝 de las señales $x_h(t)$ y  $x_k(t)$, respectivamente, determinadas mediante la transformación de Hilbert 


